# 4 - The life of a UOp tree
> From Tensor to code and result.

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [ ]:
import os
os.environ["CPU"] = "1"
os.environ["DEBUG"]="4"
os.environ["NOOPT"]="1"

from tinygrad import Tensor, dtypes, TinyJit
from tinygrad.ops import UOp, Ops, PatternMatcher, UPat, graph_rewrite

I think we are at a point where we can look in more detail at what happens to a UOp tree on its way to become the result.

We will take the `.arange()` tree from the previous chapter as an example.

We will run with `NOOPT=1` for now, and look at the optimizations in the next chapter.

Some code I quote from TinyGrad will be simplified, and it will skip some of the branches not taken. It might be useful to follow along with the actual code as you go through this chapter.

In [ ]:
a = Tensor.arange(0.5, 2, 0.2) + 1.5 # Start, stop, step => [0.5, 0.7, 0.9, 1.1, 1.3, 1.5, 1.7, 1.9] + 1.5
a.lazydata

UOp(Ops.ADD, dtypes.float, arg=None, src=(
  UOp(Ops.ADD, dtypes.float, arg=None, src=(
    UOp(Ops.RESHAPE, dtypes.float, arg=(8,), src=(
      UOp(Ops.REDUCE_AXIS, dtypes.float, arg=(Ops.ADD, (1,)), src=(
        UOp(Ops.PERMUTE, dtypes.float, arg=(1, 0), src=(
          UOp(Ops.RESHAPE, dtypes.float, arg=(8, 8), src=(
            UOp(Ops.RESHAPE, dtypes.float, arg=(8, 8, 1), src=(
              UOp(Ops.SHRINK, dtypes.float, arg=((0, 8), (0, 8)), src=(
                UOp(Ops.RESHAPE, dtypes.float, arg=(8, 16), src=(
                  UOp(Ops.SHRINK, dtypes.float, arg=((0, 128),), src=(
                    UOp(Ops.RESHAPE, dtypes.float, arg=(135,), src=(
                      UOp(Ops.EXPAND, dtypes.float, arg=(9, 15), src=(
                        UOp(Ops.RESHAPE, dtypes.float, arg=(1, 15), src=(
                          UOp(Ops.PAD, dtypes.float, arg=((7, 0),), src=(
                            UOp(Ops.EXPAND, dtypes.float, arg=(8,), src=(
                              UOp(Ops.RESH

Again, step by step.

`a.realize()`:
```py
class Tensor
    ...
    def realize(self, *lst:Tensor, do_update_stats=True) -> Tensor:
        """Triggers the computation needed to create these Tensor(s)."""
        run_schedule(*self.schedule_with_vars(*lst), do_update_stats=do_update_stats)
        return self
```

Looks like we need to call `schedule_with_vars()` on the tensor, and pass the result to `run_schedule()`. I'm not sure what's the purpose of the extra tensors (`lst`), but it's empty in out case.


```py
  def schedule_with_vars(self, *lst:Tensor) -> tuple[list[ScheduleItem], dict[Variable, int]]:
    "NOTE: A Tensor can only be scheduled once."
    big_sink = UOp.sink(*[x.lazydata for x in (self,)+lst])

    # verify Tensors match the spec
    if __debug__: type_verify(list(big_sink.toposort), tensor_uop_spec)

    schedule, var_vals, becomes_map = create_schedule_with_vars(big_sink)
    _apply_map_to_tensors(becomes_map)
    return memory_planner(schedule), var_vals
```

So, the first step is to wrap the UOp tree in a `SINK` UOp,
```py
  def sink(self, *srcs:UOp, **kwargs): return UOp(Ops.SINK, dtypes.void, (self,)+srcs, **kwargs)
```

In [ ]:
big_sink = a.lazydata.sink()
print(f"{"\n".join(str(big_sink).splitlines()[:3] + ["      ..."])}")

UOp(Ops.SINK, dtypes.void, arg=None, src=(
  UOp(Ops.ADD, dtypes.float, arg=None, src=(
    UOp(Ops.ADD, dtypes.float, arg=None, src=(
      ...


Now, we perform the `type_verify`, mentioned in the chapter on the Pattern Matcher.
`tinygrad/spec.py`:
```py
def type_verify(uops:list[UOp], *extra_specs:PatternMatcher):
  specs = [spec, *extra_specs]
  for i,u in enumerate(uops):
    spec_ret = [cast(bool|None, s.rewrite(u)) for s in specs]
    if any(ret is False for ret in spec_ret) or all(ret is None for ret in spec_ret):
      print_uops(uops)
      raise RuntimeError(f"UOp verification failed at {i} on {u.op} {u.dtype} {len(u.src)} {[x.op for x in u.src]} {u.arg}")
```

We combined the extra Pattern Matcher list(s) we get as arguments with [`spec.py:spec`](https://github.com/tinygrad/tinygrad/blob/2d6d8b735506464367b0315f9a2f424e0d19f66e/tinygrad/spec.py#L63).

Note that `specs` is a list of PatternMatchers. We don't just combine the Pattern Matchers together (`class PatternMatcher` supports `__add__`), because a Pattern Matcher will ony act on the first match, and we want to check against all specs independently.

The rules in the spec return `True`, if the UOp matched a pattern and it was deemed correct, `False` if it matched a deemed incorrect, and as always you get `None` if there was no match:

```py
spec = PatternMatcher([
  (UPat(Ops.DEFINE_GLOBAL, name="x"), lambda x: isinstance(x.dtype, (PtrDType, ImageDType)) and not x.dtype.local),
  (UPat(Ops.DEFINE_LOCAL, name="x"), lambda x: isinstance(x.dtype, PtrDType) and x.dtype.local),
  ...
])
```
Then, for each UOp we apply each spec one by one. The UOp is correct if
- There was a match in at least 1 spec that returned `True`
- There was no match with any spec that returned `False`

In [ ]:
from tinygrad.spec import spec, type_verify
from tinygrad.tensor import tensor_uop_spec
from tinygrad.ops import graph_rewrite_map, merge_views, print_uops
from tinygrad.engine.schedule import sym, reorder_view, replace_contiguous

In [ ]:
for i, u in enumerate(list(big_sink.toposort)):
    ret = [s.rewrite(u) for s in (spec, tensor_uop_spec)]
    print(f"{i:4d} {str(u.op):20s} {ret}")

   0 Ops.DEVICE           [None, True]
   1 Ops.VIEW             [True, None]
   2 Ops.CONST            [True, True]
   3 Ops.RESHAPE          [None, True]
   4 Ops.EXPAND           [None, True]
   5 Ops.PAD              [None, True]
   6 Ops.RESHAPE          [None, True]
   7 Ops.EXPAND           [None, True]
   8 Ops.RESHAPE          [None, True]
   9 Ops.SHRINK           [None, True]
  10 Ops.RESHAPE          [None, True]
  11 Ops.SHRINK           [None, True]
  12 Ops.RESHAPE          [None, True]
  13 Ops.RESHAPE          [None, True]
  14 Ops.PERMUTE          [None, True]
  15 Ops.REDUCE_AXIS      [True, None]
  16 Ops.RESHAPE          [None, True]
  17 Ops.CONST            [True, True]
  18 Ops.RESHAPE          [None, True]
  19 Ops.EXPAND           [None, True]
  20 Ops.ADD              [True, None]
  21 Ops.CONST            [True, True]
  22 Ops.RESHAPE          [None, True]
  23 Ops.EXPAND           [None, True]
  24 Ops.ADD              [True, None]
  25 Ops.SINK            

Looks good, most UOps matched with the tensor spec, some matched with the spec, and a few matched both. No match detected an error.

Now to `schedule, var_vals, becomes_map = create_schedule_with_vars(big_sink)` (simplified):

```py
def create_schedule_with_vars(big_sink:UOp) -> tuple[list[ScheduleItem], dict[Variable, int], dict[UOp, UOp]]:
  tensor_map = graph_rewrite_map(big_sink, merge_views+sym+reorder_view+replace_contiguous, ctx={})
```

graph_rewrite_map, instead of returning a new tree, returns a mapping for each UOp in the original tree:

In [ ]:
ctx={}

tensor_map = graph_rewrite_map(big_sink, merge_views+sym+reorder_view+replace_contiguous, ctx=ctx)

# +sym+reorder_view+replace_contiguous

# tensor_map = graph_rewrite_map(big_sink, sym+reorder_view+replace_contiguous, ctx=ctx)

In [ ]:
ctx

{}

In [ ]:
list(big_sink.get_children_map().keys())[3]

UOp(Ops.RESHAPE, dtypes.float, arg=(1,), src=(
  UOp(Ops.CONST, dtypes.float, arg=0.2, src=(
    UOp(Ops.VIEW, dtypes.void, arg=ShapeTracker(views=(View(shape=(), strides=(), offset=0, mask=None, contiguous=True),)), src=(
      UOp(Ops.DEVICE, dtypes.void, arg='CPU', src=()),)),)),))

In [ ]:
list(big_sink.get_children_map().values())[3]

{UOp(Ops.EXPAND, dtypes.float, arg=(8,), src=(
   UOp(Ops.RESHAPE, dtypes.float, arg=(1,), src=(
     UOp(Ops.CONST, dtypes.float, arg=0.2, src=(
       UOp(Ops.VIEW, dtypes.void, arg=ShapeTracker(views=(View(shape=(), strides=(), offset=0, mask=None, contiguous=True),)), src=(
         UOp(Ops.DEVICE, dtypes.void, arg='CPU', src=()),)),)),)),)): None}

In [ ]:
for k, v in tensor_map.items():
    print(f"{k.op} {k.arg}: {v.op}, {'[same]' if k is v else '[rewritten]'}")
    # print_uops(tuple(v.toposort))

Ops.SINK None: Ops.SINK, [rewritten]
Ops.ADD None: Ops.ADD, [rewritten]
Ops.EXPAND (8,): Ops.CONST, [rewritten]
Ops.RESHAPE (1,): Ops.CONST, [rewritten]
Ops.CONST 1.5: Ops.CONST, [same]
Ops.ADD None: Ops.ADD, [rewritten]
Ops.EXPAND (8,): Ops.CONST, [rewritten]
Ops.RESHAPE (1,): Ops.CONST, [rewritten]
Ops.CONST 0.3: Ops.CONST, [same]
Ops.RESHAPE (8,): Ops.VIEW, [rewritten]
Ops.REDUCE_AXIS (<Ops.ADD: 54>, (1,)): Ops.REDUCE_AXIS, [rewritten]
Ops.PERMUTE (1, 0): Ops.VIEW, [rewritten]
Ops.RESHAPE (8, 8): Ops.VIEW, [rewritten]
Ops.RESHAPE (8, 8, 1): Ops.VIEW, [rewritten]
Ops.SHRINK ((0, 8), (0, 8)): Ops.VIEW, [rewritten]
Ops.RESHAPE (8, 16): Ops.VIEW, [rewritten]
Ops.SHRINK ((0, 128),): Ops.VIEW, [rewritten]
Ops.RESHAPE (135,): Ops.VIEW, [rewritten]
Ops.EXPAND (9, 15): Ops.VIEW, [rewritten]
Ops.RESHAPE (1, 15): Ops.VIEW, [rewritten]
Ops.PAD ((7, 0),): Ops.VIEW, [rewritten]
Ops.EXPAND (8,): Ops.CONST, [rewritten]
Ops.RESHAPE (1,): Ops.CONST, [rewritten]
Ops.CONST 0.2: Ops.CONST, [same]
Ops.VI

In [ ]:
for k, v in tensor_map.items():
    print(f"{k.op} -> {v.op}")


Ops.SINK -> Ops.SINK
Ops.ADD -> Ops.ADD
Ops.EXPAND -> Ops.CONST
Ops.RESHAPE -> Ops.CONST
Ops.CONST -> Ops.CONST
Ops.ADD -> Ops.ADD
Ops.EXPAND -> Ops.CONST
Ops.RESHAPE -> Ops.CONST
Ops.CONST -> Ops.CONST
Ops.RESHAPE -> Ops.VIEW
Ops.REDUCE_AXIS -> Ops.REDUCE_AXIS
Ops.PERMUTE -> Ops.VIEW
Ops.RESHAPE -> Ops.VIEW
Ops.RESHAPE -> Ops.VIEW
Ops.SHRINK -> Ops.VIEW
Ops.RESHAPE -> Ops.VIEW
Ops.SHRINK -> Ops.VIEW
Ops.RESHAPE -> Ops.VIEW
Ops.EXPAND -> Ops.VIEW
Ops.RESHAPE -> Ops.VIEW
Ops.PAD -> Ops.VIEW
Ops.EXPAND -> Ops.CONST
Ops.RESHAPE -> Ops.CONST
Ops.CONST -> Ops.CONST
Ops.VIEW -> Ops.VIEW
Ops.DEVICE -> Ops.DEVICE



```

  # get realizes
  sink = tensor_map[big_sink]
  realize_map = group_realizes(sink)

  # merge_views + create_kernels
  kernel_map = graph_rewrite_map(sink, merge_views+create_kernels, ctx=KernelContext(realize_map, ops_metadata), bottom_up=True)

  sched_sink = kernel_map[sink]

  type_verify(list(sched_sink.toposort), kernel_spec)

  # map tensors to buffer/const, optionally apply a VIEW on top
  becomes_map: dict[UOp, UOp] = {}
  for k,v in tensor_map.items():
    # ASSIGN always becomes the target buffer
    if v.op is Ops.ASSIGN: becomes_map[k] = v.src[0]
    # if we created a new buffer for this tensor, map it to the assigned buffer
    elif (a:=kernel_map.get(v.base)) is not None and (a:=a.base).op is Ops.ASSIGN:
      becomes_map[k] = a.src[0] if a.src[0].st == v.st else a.src[0].view(unwrap(v.st))
    # tensors can also simplify to an existing buffer/const
    else:
      if k is v: continue
      if v.base.op is Ops.BUFFER: becomes_map[k] = v
      if v.base.op is Ops.CONST and all_int(v.shape): becomes_map[k] = v

  # if a kernel depends on a buffer, and that buffer is later assigned to, make the assign depend on the kernel's assign
  kernel_assign: dict[UOp, UOp] = {}
  assign_rep: dict[UOp, UOp] = {}
  for u in sched_sink.toposort:
    if u.op is not Ops.ASSIGN: continue
    kernel_assign[u.buf_uop] = u
    for s in u.src[1].src:
      if s.op is not Ops.BUFFER or s is u.buf_uop or (a:=kernel_assign.get(s)) is None: continue
      if any(x.op is Ops.ASSIGN and x.buf_uop is s for x in u.toposort):
        raise RuntimeError(f"cycle detected in graph, kernel for {u.buf_uop} must either depend on ASSIGN or BUFFER")
      assign_rep[a] = kernel_assign[s] = a.replace(src=a.src+(u,))
  if assign_rep:
    sched_sink = sched_sink.substitute(assign_rep)
    type_verify(list(sched_sink.toposort), kernel_spec)

  # display the final graph
  if getenv("VIZ"): graph_rewrite(sched_sink, PatternMatcher([]), name="View Kernel Graph")
  if getenv("VIZ"): graph_rewrite(sched_sink, PatternMatcher([]), name="View Memory Graph")

  # final toposort (bfs)
  children: dict[UOp, list[UOp]] = {}
  in_degree: dict[UOp, int] = {}
  for u in sched_sink.toposort:
    if u.op is not Ops.ASSIGN: continue
    in_degree[u] = 0
    for s in u.src[1].src:
      if s.op is not Ops.ASSIGN: continue
      children.setdefault(s, []).append(u)
      in_degree[u] += 1

  queue = deque(k for k,v in in_degree.items() if v == 0)
  schedule: list[ScheduleItem] = []
  var_vals: dict[Variable, int] = {}
  while queue:
    u = queue.popleft()
    # TODO: move this to create_kernels
    k = fix_kernel_ast(u.src[1], var_vals)
    schedule.append(ScheduleItem(k.arg.ast, tuple(s.buf_uop.buffer for s in k.src), k.arg.metadata))
    for x in children.get(u, []):
      in_degree[x] -= 1
      if in_degree[x] == 0: queue.append(x)

  # confirm everything was scheduled correctly
  if len(schedule) != (kc:=len(in_degree)): raise RuntimeError(f"cycle detected in graph, created {kc} kernels but only scheduled {len(schedule)}")
  if DEBUG >= 1 and len(schedule) >= 10: print(f"scheduled {len(schedule)} kernels")
  # capture process replay
  if CAPTURE_PROCESS_REPLAY:
    with Context(PICKLE_BUFFERS=0): PROCESS_REPLAY_CAPTURE[str(big_sink.key)] = pickle.dumps((big_sink, ContextVar._cache, [x.ast for x in schedule]))
  return schedule, var_vals, becomes_map


```